In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("galaxy_users.csv", encoding="utf-8")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

## Q1.

In [4]:
df_q1 = df[["customerID", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]].copy() # 7032
df_q1 = df_q1[df_q1.iloc[:, 1:].apply(lambda x: x.str.contains("^Yes$|^No$").sum(), axis=1) == 6 ] #5512
# apply(lambda x: x.isin(["Yes", "No"]).sum(), axis = 1) 과 같이 편리하게 처리 가능
df_q1["x"] = df_q1.iloc[:,1:].apply(lambda x: x.str.contains("Yes").sum(), axis=1)
service_6 = len(df_q1[df_q1["x"] == 6])
service_1 = len(df_q1[df_q1["x"] == 1])

round(service_1 / service_6, 1), service_6, service_1

(3.4, 284, 966)

In [5]:
#Q1. 정답
df.columns # 이거 출력한 후에 순서가 맞을 수 있은
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].copy()
df_q1["OnlineSecurity"].unique() # 'No internet service' 가 포함된 것을 알 수 있따.
# 전체에 대해서 알아보자.
for col_name in df_q1:
    print(col_name, df_q1[col_name].unique())
# 더 쉬운방법 
df_q1.apply(lambda x : x.unique())
# 시험버전처럼 낮을 경우
df_q1.apply(lambda x: [x.unique()])

#아무튼 결론적으로 'No internet service' 가 있다.
#아래처럼 제거할 수 있겠군.
df_q1_sub = df_q1.loc[df_q1["OnlineSecurity"] != "No internet service", ]
# df_q1.loc[df_q1["OnlineSecurity"] == "No internet service", ]

# 만약 No internet service 가 여기저기 막 들어있ㅇ면 반복문 활용해야 하고,
# 결측치를 활용한 처리는 사전에 결측치가 모두 제거된 상태에서 실시해야 한다.
df_q1_sub = df_q1.replace("No internet service", np.nan).dropna() # 5512 개 나옴
df_q1_sub
df_q1_sub = df_q1_sub.replace({"Yes": 1, "No": 0})
df_q1_sub["cnt"] = df_q1_sub.sum(axis = 1)
ser_cnt = df_q1_sub["cnt"].value_counts()
round(ser_cnt[1] / ser_cnt[6], 1)



OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['No' 'Yes' 'No internet service']
StreamingTV ['No' 'Yes' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']


3.4

In [19]:
# 만약 No internet service 말고 다른 이것저것들이 있어 yes no 를 제외하고 모두 치환하는 경우는 참고로 
df_d = pd.read_csv("../01_DX_Class_Data_Set/diamonds.csv")
df_d[["cut", "color", "clarity"]].apply(lambda x : x.unique())



cut         [Ideal, Premium, Good, Very Good, Fair]
color                         [E, I, J, H, F, G, D]
clarity    [SI2, SI1, VS1, VS2, VVS2, VVS1, I1, IF]
dtype: object

In [22]:
# H와 F를 제외한 나머지를 모두 -1로 치환하자.
ser_u = df_d[["cut", "color", "clarity"]].apply(lambda x: x.unique()).explode()
ser_u

cut            Ideal
cut          Premium
cut             Good
cut        Very Good
cut             Fair
color              E
color              I
color              J
color              H
color              F
color              G
color              D
clarity          SI2
clarity          SI1
clarity          VS1
clarity          VS2
clarity         VVS2
clarity         VVS1
clarity           I1
clarity           IF
dtype: object

In [27]:
ser_repl = pd.Series(np.where(~ser_u.isin(["H", "F"]), -1, ser_u), index = ser_u)
ser_repl.to_dict()

{'Ideal': -1,
 'Premium': -1,
 'Good': -1,
 'Very Good': -1,
 'Fair': -1,
 'E': -1,
 'I': -1,
 'J': -1,
 'H': 'H',
 'F': 'F',
 'G': -1,
 'D': -1,
 'SI2': -1,
 'SI1': -1,
 'VS1': -1,
 'VS2': -1,
 'VVS2': -1,
 'VVS1': -1,
 'I1': -1,
 'IF': -1}

In [28]:
df_d2 = df_d.replace(ser_repl)
df_d2[["cut", "color", "clarity"]].apply(lambda x: x.unique()) 
#모두 치환 완료!

cut              [-1]
color      [-1, H, F]
clarity          [-1]
dtype: object

## Q2.

In [7]:
df_q2 = df[["TotalCharges", "tenure", "MonthlyCharges"]].copy()
df_q2["x"] = (df["TotalCharges"] // df["MonthlyCharges"]).astype("float")
df_q2.iloc[:, 1:].corr()

round(0.998831, 3)
#0.999

0.999

In [8]:
## Q2. 정답
df_q2 = df[["tenure", "MonthlyCharges", "TotalCharges"]].copy()
df_q2["month"] = df_q2["TotalCharges"] // df_q2["MonthlyCharges"]
df_q2.iloc[:, [0,1,3]].corr().round(3)

,tenure,MonthlyCharges,month
tenure,1.000,0.247,0.999
MonthlyCharges,0.247,1.000,0.246
month,0.999,0.246,1.000


## Q3.

In [25]:
## Q3 정답
df_q3 = df[["Churn", "SeniorCitizen", "Partner", "Dependents", "tenure", "MonthlyCharges", "TotalCharges", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingMovies", "PaperlessBilling"]]
df_q3 = df_q3.replace("^Yes$", 1, regex=True).replace("^No$", 0, regex=True).replace("[^01]", -1, regex=True)

train, test = train_test_split(df_q3, test_size=0.3, random_state=123)
len(train)

scaler = MinMaxScaler()
train_norm = scaler.fit_transform(train)
test_norm = scaler.transform(test)

model = LogisticRegression(random_state=123)
model.fit(X=train_norm[:, 1:], y=train_norm[:, 0])
Y_pred = model.predict(X=test_norm[:, 1:])
result = f1_score(y_true=test_norm[:,0], y_pred=Y_pred)
round(result, 2)

0.55

In [17]:

col1 = ["SeniorCitizen", "Partner", "Dependents", "tenure", "MonthlyCharges", "TotalCharges"]
col2 = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingMovies", "PaperlessBilling"]
df_q3 = df[["Churn"] + col1 + col2].copy()

df_q3 = df_q3.replace({"Yes":1, "No":0}) 
df_q3.dtypes # object가 섞여있으니 다 바뀐건 아닌것을 알 수 있다. (No internet service 가 있으니 당연한 것)

df_q3_obj = df_q3.select_dtypes(exclude = "number") # 시험버전에서는 사용이 불가능하다고 한다.
ser_dtypes = df_q3.dtypes
obj_index = ser_dtypes[ser_dtypes == "object"].index
df_q3_obj = df_q3[obj_index]

df_q3_obj.apply(lambda x : x.unique()) # No internet service가 있음을 확인가능

df_q3 = df_q3.replace("No internet service", -1)
df_q3
df_train ,df_test = train_test_split(df_q3, train_size=0.7, random_state=123)
print(len(df_train))

model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor = model_nor.transform(df_test)

model_nor.data_max_

model_lr = LogisticRegression(random_state=123)
model_lr.fit(X=arr_train_nor[:, 1:], y=arr_train_nor[:, 0])
pred = model_lr.predict(arr_test_nor[:, 1:])
pred[:3]
round(f1_score(y_true = arr_test_nor[: ,0], y_pred=pred), 2)


4922


0.55